#### **Contextual Chunk Headers**

In this approach we give each chunk a Title or a brief header information (that can represent true content of that chunk)

This feature leads to enhancement in retrieval quality.

##### **Flow**
- Create chunks from document
- Create title corresponding to each chunk using LLM
- Add those titles with chunks and create embeddings and store them in vectorstore
- Create a retriever | make a RAG flow | test your RAG
- You can do a performance test for retriever with CCH and without CCH


